In [10]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import requests
import collection.connect_mongodb as connect_mongodb
import os
import re
import json
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import urllib.parse
import random
import concurrent.futures
import xml.etree.ElementTree as ET
import time
from pymongo import errors
from IPython.display import clear_output

In [11]:
def update_json(file_path, new_data):
    # Kiểm tra xem tệp có tồn tại không
    if os.path.exists(file_path):
        # Đọc dữ liệu hiện có từ tệp
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                existing_data = json.load(file)
            except json.JSONDecodeError:
                existing_data = {}
    else:
        existing_data = {}

    # Kiểm tra nếu dữ liệu hiện có không phải là dict, biến nó thành dict
    if not isinstance(existing_data, dict):
        existing_data = {}

    existing_data.update(new_data)    
    

    # Ghi dữ liệu đã cập nhật trở lại tệp
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)

In [12]:
def translate(text, src_lang='auto', dest_lang='en'):
    # URL của Google Translate với các tham số truy vấn
    url = f"https://translate.google.com/m?hl={dest_lang}&sl={src_lang}&q={text}"
    
    # Gửi yêu cầu GET tới Google Translate
    response = requests.get(url)
    
    # Kiểm tra xem yêu cầu có thành công không
    if response.status_code == 200:
        # Phân tích cú pháp HTML của trang
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Tìm phần tử chứa kết quả dịch
        translated_text = soup.find('div', class_='result-container').text
        
        return translated_text
    else:
        return None

In [13]:
product_name_dicts ={}
product_name_lost_other_errors={}
product_name_lost_404={}
link_special = {}
# Đường dẫn tới file JSON
file_path = 'view_product_id/json_product_name_view.json'

# Đọc nội dung của file JSON
with open(file_path, 'r', encoding='utf-8') as file:
    try:
        product_name_dicts = json.load(file)
    except:
        product_name_dicts = {}

# Biến 'data' bây giờ chứa dữ liệu từ file JSON
print(len(product_name_dicts))

file_path = 'view_product_id/json_product_name_view_lost_other_errors.json'

# Đọc nội dung của file JSON
with open(file_path, 'r', encoding='utf-8') as file:
    try:
        product_name_lost_other_errors = json.load(file)
    except:
        product_name_lost_other_errors = {}

# Biến 'data' bây giờ chứa dữ liệu từ file JSON
print(len(product_name_lost_other_errors))

file_path = 'view_product_id/json_product_name_view_lost_404.json'

# Đọc nội dung của file JSON
with open(file_path, 'r', encoding='utf-8') as file:
    try:
        product_name_lost_404 = json.load(file)
    except:
        product_name_lost_404 = {}

# Biến 'data' bây giờ chứa dữ liệu từ file JSON
print(len(product_name_lost_404))

file_path = 'view_product_id/json_link_special_view.json'

# Đọc nội dung của file JSON
with open(file_path, 'r', encoding='utf-8') as file:
    try:
        link_special = json.load(file)
    except:
        link_special={}
# Biến 'data' bây giờ chứa dữ liệu từ file JSON
print(len(link_special))


15802
0
1147
65


In [14]:
def get_product_name__glamira(url_link,product_id):
    global product_name_dicts
    global product_name_lost_other_errors
    global product_name_lost_404
    global link_special
    try:    
        session = requests.Session()
        retry_strategy = requests.packages.urllib3.util.retry.Retry(
            total=3,  # Number of retries
            status_forcelist=[429, 500, 502, 503, 504],  # HTTP statuses to retry on
            allowed_methods=["HEAD", "GET", "OPTIONS"],  # Methods to retry
        )
        adapter = requests.adapters.HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)
        session.mount("http://", adapter)

        api_response = session.get(url_link, timeout=10)
        api_response.raise_for_status()  # Raise HTTPError for bad responses 
        if api_response.status_code == 200:
            content = api_response.content
            soup = BeautifulSoup(content,'html.parser')
            try:
                product_name = soup.find('span',class_='base').text
            except AttributeError:
                print(f"link special:f{url_link}")
                link_special[product_id]=url_link
                update_json(r'view_product_id/json_link_special_view.json',link_special)
                return {}
            
            translated_product_name = translate(product_name)
            if translated_product_name == None:
                print(f"Failed to translate data from {url_link}")
                product_name_lost_other_errors[product_id]=url_link    
                update_json(r'view_product_id/json_product_name_view_lost_other_errors.json',product_name_lost_other_errors) 
                return {}
            
            product_name_dicts[product_id] = translated_product_name
            update_json(r'view_product_id/json_product_name_view.json',product_name_dicts)
            return product_name_dicts
        else:
            print(f"Failed to fetch data from {url_link}. Status code: {api_response.status_code}")
            product_name_lost_other_errors[product_id]=url_link    
            update_json(r'view_product_id/json_product_name_view_lost_other_errors.json',product_name_lost_other_errors)
            return {}
    except requests.RequestException as e:
        try:
            if e.response.status_code ==404:
                print(f"Error 404:Not Found-{url_link}")
                product_name_lost_404[product_id]=url_link 
                update_json(r'view_product_id/json_product_name_view_lost_404.json',product_name_lost_404)
            else:
                print(f"Others Errors: {url_link}")
                print(f"status_code:{api_response.status_code}")
                product_name_lost_other_errors[product_id]=url_link    
                update_json(r'view_product_id/json_product_name_view_lost_other_errors.json',product_name_lost_other_errors)
                return {} 
        except Exception as e:
            print(f"link special:f{url_link}")
            product_name_lost_other_errors[product_id]=url_link
            update_json(r'view_product_id/json_product_name_view_lost_other_errors.json',product_name_lost_other_errors)
            return {}

In [15]:
def get_product_name__ring_paare(url_link,product_id):
    global product_name_dicts
    global product_name_lost_other_errors
    global product_name_lost_404
    global link_special
    try:    
        session = requests.Session()
        retry_strategy = requests.packages.urllib3.util.retry.Retry(
            total=3,  # Number of retries
            status_forcelist=[429, 500, 502, 503, 504],  # HTTP statuses to retry on
            allowed_methods=["HEAD", "GET", "OPTIONS"],  # Methods to retry
        )
        adapter = requests.adapters.HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)
        session.mount("http://", adapter)

        api_response = session.get(url_link, timeout=10)
        api_response.raise_for_status()  # Raise HTTPError for bad responses 
        if api_response.status_code == 200:
            content = api_response.content
            soup = BeautifulSoup(content,'html.parser')
            try:
                product_name = soup.find('div',class_ = "product-info-desc").text
            except AttributeError:
                print(f"link special:f{url_link}")
                link_special[product_id]=url_link
                update_json(r'view_product_id/json_link_special_view.json',link_special)
                return {}
            
            translated_product_name = translate(product_name)
            if translated_product_name == None:
                print(f"Failed to translate data from {url_link}")
                product_name_lost_other_errors[product_id]=url_link    
                update_json(r'view_product_id/json_product_name_view_lost_other_errors.json',product_name_lost_other_errors) 
                return {}
            
            product_name_dicts[product_id] = translated_product_name
            update_json(r'view_product_id/json_product_name_view.json',product_name_dicts)
            return product_name_dicts
        else:
            print(f"Failed to fetch data from {url_link}. Status code: {api_response.status_code}")
            product_name_lost_other_errors[product_id]=url_link    
            update_json(r'view_product_id/json_product_name_view_lost_other_errors.json',product_name_lost_other_errors)
            return {}
    except requests.RequestException as e:
        try:
            if e.response.status_code ==404:
                print(f"Error 404:Not Found-{url_link}")
                product_name_lost_404[product_id]=url_link 
                update_json(r'view_product_id/json_product_name_view_lost_404.json',product_name_lost_404)
            else:
                print(f"Others Errors: {url_link}")
                print(f"status_code:{api_response.status_code}")
                product_name_lost_other_errors[product_id]=url_link    
                update_json(r'view_product_id/json_product_name_view_lost_other_errors.json',product_name_lost_other_errors)
                return {} 
        except Exception as e:
            print(f"link special:f{url_link}")
            product_name_lost_other_errors[product_id]=url_link
            update_json(r'view_product_id/json_product_name_view_lost_other_errors.json',product_name_lost_other_errors)
            return {}

In [16]:
def main_code():
    collection =connect_mongodb.connect_db()
    documents = collection.find({}, {"viewing_product_id":1,'current_url': 1, '_id': 0})
    doc = 0
    for document in documents:
        doc+=1
        print(f"document:{doc}")
        if(doc%500000==0):
            print("Sleep 5s")
            time.sleep(5)
            clear_output(wait=True)
        view_product_id = document.get('viewing_product_id')
        if view_product_id not in product_name_dicts and view_product_id not in product_name_lost_404  :
            current_url = document.get('current_url')
            if 'viewing_product_id' in document and  'www.glamira' in current_url:
                print(current_url)
                print(view_product_id)
                get_product_name__glamira(current_url,view_product_id)
            elif 'viewing_product_id' in document  and 'www.ring-paare' in current_url:
                print(current_url)
                print(view_product_id)
                get_product_name__ring_paare(current_url,view_product_id)    

            print(f"product_dict_view_len: {len(product_name_dicts)}")
            
        print("--------------------------------------------------------")
    # with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    #     futures = [executor.submit(get_product_name,document.get('current_url'),document.get('product_id')) for document in documents if  'product_id' in document]
        
    #     for future in concurrent.futures.as_completed(futures):
    #         result = future.result()
    #         print(f"len of product dictionary:{len(product_name_dicts)}")

In [17]:
def test():
    collection =connect_mongodb.connect_db()
    documents = collection.find({}, {"viewing_product_id":1,'current_url': 1, '_id': 0})
    doc = 0
    for document in documents:
        doc+=1
        if(doc%500000==0):
            print(doc)
            time.sleep(5)
        if 'viewing_product_id' in document:
            product_id = document.get('viewing_product_id')
            if product_id not in product_name_dicts and product_id not in product_name_lost_404 and product_id not in link_special:
                url_link = document.get('current_url')
                print(product_id)
                print(url_link)
                link_special[product_id]=url_link
                update_json(r'view_product_id/json_link_special_view.json',link_special)
                print("---------------------------------")

In [ ]:
# main_code()

In [18]:
test()

# while True:
#     try:
#         main_code()
#     except errors.CursorNotFound as e:
#         print(f"Cursor not found:{e}")
#         time.sleep(10)
#         continue
#     except Exception as e:
#         print(f"An expected error occurred:{e}")
#         time.sleep(5)
#         continue
#     break

500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
10000000
10500000
11000000
11500000
12000000
12500000
13000000
13500000
14000000
14500000
15000000
15500000
16000000
16500000
17000000
17500000
18000000
18500000
19000000
19500000
20000000
20500000
21000000
21500000
22000000
22500000
23000000
23500000
24000000
24500000
25000000
25500000
26000000
26500000
27000000
27500000
28000000
28500000
29000000
29500000
30000000
30500000
31000000
31500000
32000000
32500000
33000000
33500000
34000000
34500000
35000000
35500000
36000000
36500000
37000000
37500000
38000000
38500000
39000000
39500000
40000000
40500000
41000000
